# お薦め商品の提案（Association分析）

## 処理 Pattern と想定される業務利用 Sean
### Association 分析
- 大量の商品購入履歴 Data から「商品Ａと商品Ｂが同時に購入されることが多い」といった法則を見つけ出す分析手法。
- 「商品Ａと商品Ｂを Set にして割り引き価格を設定する」といった Marketing施策を導く
- 単価が少額の商品Ｘの仕入れを継続するか検討する時、分析の結果、高単価の商品Ｙと Set で売れており売上に貢献していることが判明。商品Ｘの仕入れを継続する判断を導く

## 教師なし学習（分析）の処理 Pattern
単に分析結果を出しただけでは、AI の効果がでたことにならない。
**得られた知見を基に、何か戦略を考え、その効果があって初めて分析の意味を持つ**

### 教師あり学習との違い
| 学習方法   | 特徴                                                                                           |
|--------|----------------------------------------------------------------------------------------------|
| 教師あり学習 | 正解 Data の収集が大変だが、Model構築までできれば業務での活用は比較的簡単なことが多い。                                            |
| 教師なし学習 | 正解 Data の収集は不要なので Model構築は比較的簡単。しかし、業務での活用は「知見を基に戦略を考え、それが効果を出す」と1 Action 余分に必要なので比較的難易度が高い。 |

## 例題の Data 説明と Use case

Data: UCI Dataset の「Online Retail Data Set 」（Online 小売り Data Set）を利用。
イギリスを拠点とし、Gift 商品を中心に販売している EC Site の取引履歴 Data
- 取引はまとまった単位で行なわれているので、一般顧客より小売業者が主な対象と考えられる。
- Data 件数は総数で 54万件
- 商品購買分析の例題でよく利用される Data.

この Data からフランスの顧客からよく Set 購入されていることが商品を見つけだす。

## Model の概要
Association分析で重要な概念
- 支持度（support）
- 確信度（confidence)
- Lift値（lift）

#### 支持度（support）
計算の出発点になる指標値。
**顧客全体の中で、今注目している商品を購入した顧客の比率**
S = A/n
S = B/n

#### 確信度（confidence）
Ａを買った人を全体とした時、そのうち何人がＢも一緒に購入したか
S(A&B) / S(A)

#### Lift値
S(A&B) / S(A)*S(B)
ＡとＢが独立な状態であれば Lift 値は１になる。逆に相関の関係があると Lift値は１より大きくなる。
Lift値が高ければ高いほどＡとＢの相関は強い、ということになる。

計算方法自体は、Simple だが実際には何万点、何十万点にも件数がなりえ、そうなると組み合わせの数が爆発して有限時間で計算できなくなる。
そこで、「**支持度の閾値**」という概念がでてくる。

### Association分析
1. 探索開始時に支持度（対象事象の購入比率の下限値）を定める。
2. この下限値より下の事象は、探索の対象から切り捨てる。（**アプリオリ分析**）
3. 抽出した商品List から確信度、または Lift値の高い Rule を抽出する。

購買履歴 -> アプリオリ分析（支持度で抽出）-> 商品List -> Rule抽出（Lift値 or 確信度で抽出）-> Association Rule

アプリオリ分析の仕掛けがあるので、対象商品が沢山あっても有限時間で計算できる。
一方、めったに売れない商品で支持度が低い場合、切り捨てられて、計算対象に入らなくなる。このため **「支持度の下限値」が Association分析で最も重要な Parameter** になる。